LIBRERIAS:

In [8]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

Carga de los datos:

In [11]:
movies_data = r'C:\Users\EFRAIN LINARES\Desktop\Proyecto 1\Datos_clean\movies_clean.parquet'
credit_crew_data = r'C:\Users\EFRAIN LINARES\Desktop\Proyecto 1\Datos_clean\credit_crew_clean.parquet'
credit_cast_data = r'C:\Users\EFRAIN LINARES\Desktop\Proyecto 1\Datos_clean\credit_cast_clean.parquet'

df_movies = pd.read_parquet(movies_data)
df_crew = pd.read_parquet(credit_crew_data)
df_cast = pd.read_parquet(credit_cast_data)

In [12]:
df_movies

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,release_year,return,genres_name,production_companies_name,production_countries_name,spoken_languages_name
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415,1995,12.451801,Animation,Pixar Animation Studios,United States of America,English
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415,1995,12.451801,Comedy,Pixar Animation Studios,United States of America,English
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Released,Toy Story,7.7,5415,1995,12.451801,Family,Pixar Animation Studios,United States of America,English
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,6.9,2413,1995,4.043035,Adventure,TriStar Pictures,United States of America,English
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Released,Jumanji,6.9,2413,1995,4.043035,Adventure,Teitler Film,United States of America,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134323,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0,104.0,Released,Robin Hood,5.7,26,1991,0.000000,Romance,CanWest Global Communications,Canada,English
134324,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0,360.0,Released,Century of Birthing,9.0,3,2011,0.000000,Drama,Sine Olivia,Philippines,
134325,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90.0,Released,Betrayal,3.8,6,2003,0.000000,Action,American World Pictures,United States of America,English
134326,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90.0,Released,Betrayal,3.8,6,2003,0.000000,Drama,American World Pictures,United States of America,English


In [13]:
df_movies = df_movies[df_movies['release_year'] >= 1980]

In [14]:
principales_idiomas = ['English', 'Français', 'Deutsch', 'Español', 'Italiano']

df_movies = df_movies[df_movies['spoken_languages_name'].isin(principales_idiomas)]

In [15]:
paises_principales = ['United States of America', 'United Kingdom', 'France', 'Canada', 
                      'Japan', 'Germany', 'Italy', 'Russia', 'India', 'Spain', 'Argentina']

df_movies = df_movies[df_movies['production_countries_name'].isin(paises_principales)]

Transformaciones previas a unir los Dataframes:

Los valores de las columnas "genres_name", "crew_name" y "cast_name" se convierten en listas, dejando solo un valor de ID en cada una de ellas.

In [16]:
genero = (df_movies[['id', 'genres_name']]
                               .drop_duplicates()
                               .groupby('id')['genres_name']
                               .apply(list)
                               .reset_index(name='generos'))

In [17]:
directores = (df_crew[['id', 'crew_name']]
                               .drop_duplicates()
                               .groupby('id')['crew_name']
                               .apply(list)
                               .reset_index(name='directores'))

In [18]:
actores = (df_cast[['id', 'cast_name']]
                               .drop_duplicates()
                               .groupby('id')['cast_name']
                               .apply(list)
                               .reset_index(name='actores'))

In [19]:
col_movies = df_movies[['id', 'title','overview']]
col_genero = genero[['id', 'generos']]
col_directores = directores[['id', 'directores']]
col_actores = actores[['id', 'actores']]

# Realizar merge secuencial para combinar los DataFrames
df_tags = pd.merge(col_movies, col_genero, on='id', how='inner')
df_tags = pd.merge(df_tags, col_directores, on='id', how='inner')
df_tags = pd.merge(df_tags, col_actores, on='id', how='inner')

In [20]:
df_tags

,id,title,overview,generos,directores,actores
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]",[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney]"
1,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]",[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney]"
2,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]",[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney]"
3,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]",[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
4,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]",[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
...,...,...,...,...,...,...
88012,30840,Robin Hood,"Yet another version of the classic epic, with ...","[Drama, Action, Romance]",[John Irvin],"[Patrick Bergin, Uma Thurman, David Morrissey,..."
88013,30840,Robin Hood,"Yet another version of the classic epic, with ...","[Drama, Action, Romance]",[John Irvin],"[Patrick Bergin, Uma Thurman, David Morrissey,..."
88014,67758,Betrayal,"When one of her hits goes wrong, a professiona...","[Action, Drama, Thriller]",[Mark L. Lester],"[Erika Eleniak, Adam Baldwin, Julie du Page, J..."
88015,67758,Betrayal,"When one of her hits goes wrong, a professiona...","[Action, Drama, Thriller]",[Mark L. Lester],"[Erika Eleniak, Adam Baldwin, Julie du Page, J..."


In [21]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88017 entries, 0 to 88016
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          88017 non-null  int64 
 1   title       88017 non-null  object
 2   overview    88017 non-null  object
 3   generos     88017 non-null  object
 4   directores  88017 non-null  object
 5   actores     88017 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.0+ MB


Transformaciones

La columna 'overview' será transformada para dividir el texto en caracteres individuales. Eliminamos cualquier espacio que pueda existir. Este paso es crucial para la vectorización. Se crea la columna 'tags' para consolidar los resultados de las otras columnas, asegurándonos de tener solo uno por ID.

In [22]:
#Se Separas en caracteres a la columna de 'overview'
#Se eliminan las filas vacías que puedan haber quedado.. 
df_tags.dropna(subset=['overview'], inplace=True)
df_tags['overview'] = df_tags['overview'].apply(lambda x:x.split())

In [23]:
#Se ha creado una función para eliminar los espacios que puedan existir.
def eliminar_espacios(cadena):
    sin_espacios = []
    for i in cadena:
        sin_espacios.append(i.replace(" ", ""))
    return sin_espacios

#Se aplica la función creada para eliminar espacios en las columnas 'gener_name', 'crew_name' y 'cast_name'.
df_tags['generos'] = df_tags['generos'].apply(eliminar_espacios)
df_tags['directores'] = df_tags['directores'].apply(eliminar_espacios)
df_tags['actores'] = df_tags['actores'].apply(eliminar_espacios)

In [24]:
df_tags['etiquetas'] = df_tags['overview'] + df_tags['generos'] + df_tags['directores'] + df_tags['actores']

In [25]:
#Eliminamos las columnas que ya no nos sirven.
df_tags = df_tags.drop(columns = ['overview', 'generos', 'directores', 'actores','id'])

In [26]:
#la línea de código convierte cada lista de strings en la columna 'etiquetas' del DataFrame df_tags en una cadena única de texto, donde cada elemento está separado por espacios, 
# utilizando una función lambda con apply().
df_tags['etiquetas'] = df_tags ['etiquetas'].apply(lambda x: " ".join(x))

In [27]:
df_tags = df_tags.drop_duplicates()

In [28]:
df_tags

,title,etiquetas
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
3,Jumanji,When siblings Judy and Peter discover an encha...
12,Grumpier Old Men,A family wedding reignites the ancient feud be...
16,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
19,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...
87995,The Morning After,The Morning After is a feature film that consi...
87999,The Burkittsville 7,A film archivist revisits the story of Rustin ...
88001,Caged Heat 3000,It's the year 3000 AD. The world's most danger...
88002,Robin Hood,"Yet another version of the classic epic, with ..."


In [29]:
stop_words = set(stopwords.words('english'))

Creación de Modelo de Recomendación

In [30]:
#Reiniciar el index porque me estaban dando mal las recomendaciones. 
df_tags.reset_index(drop=True, inplace=True)

In [31]:
cv = CountVectorizer(max_features = 10000 , stop_words= 'english')

In [32]:
vector = cv.fit_transform(df_tags['etiquetas']).toarray()

In [34]:
vector.shape

(15629, 10000)

In [35]:
similitud_coseno = cosine_similarity(vector)

similitud_coseno

array([[1.        , 0.03892495, 0.04433511, ..., 0.        , 0.        ,
        0.        ],
       [0.03892495, 1.        , 0.07118685, ..., 0.03965258, 0.        ,
        0.        ],
       [0.04433511, 0.07118685, 1.        , ..., 0.        , 0.03001501,
        0.        ],
       ...,
       [0.        , 0.03965258, 0.        , ..., 1.        , 0.02507849,
        0.06302535],
       [0.        , 0.        , 0.03001501, ..., 0.02507849, 1.        ,
        0.08377078],
       [0.        , 0.        , 0.        , ..., 0.06302535, 0.08377078,
        1.        ]])

In [36]:
def recomendacion(titulo):
    try:
        indice = df_tags[df_tags['title'] == titulo].index[0]
    
        distancia = sorted(list(enumerate(similitud_coseno[indice])), reverse = True, key = lambda x: x[1])

        recomendadas = [df_tags.iloc[i[0]].title for i in distancia[1:6]]
        
        print(f"Porque viste '{titulo}', tal vez te guste:")
        for pelicula in recomendadas:
            print(pelicula)
    except IndexError:
        print(f"No se encontró la película '{titulo}' en la base de datos.")

In [37]:
# Mostrar los nombres de las columnas en el DataFrame
print(df_tags.columns)

Index(['title', 'etiquetas'], dtype='object')


In [38]:
recomendacion("Toy Story")

Porque viste 'Toy Story', tal vez te guste:
Toy Story 3
Toy Story 2
Small Fry
The 40 Year Old Virgin
Andy Peters: Exclamation Mark Question Point


In [39]:
#Descargamos la base de datos que utilizamos para crear la función de recomendación. 
df_tags.to_parquet('tags_ML.parquet', index=False)